[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CLDiego/SPE_GeoHackathon_2025/blob/main/S1_M1_LLM_HF.ipynb)

***
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/write.svg" width="20"/> Follow along by running each cell in order
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/code.svg" width="20"/> Make sure to run the environment setup cells first
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/reminder.svg" width="20"/> Wait for each installation to complete before proceeding
- <img src="https://github.com/CLDiego/uom_fse_dl_workshop/raw/main/figs/icons/list.svg" width="20" /> Don't worry if installations take a while - this is normal!

In [ ]:
# Environment setup [If running outside Colab]
# !pip install transformers torch matplotlib plotly scikit-learn ipython

# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# Hugging Face API token
# Retrieving the token is required to get access to HF hub
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Session 01 // Module 01: Large Language Models (LLMs) with HuggingFace

In this module, we'll explore the fundamentals of Large Language Models (LLMs) using HuggingFace transformers. We'll cover tokens, embeddings, context windows, and hands-on text generation with a focus on geoscience applications.

## Learning Objectives
- Understand what tokens, embeddings, and context windows are
- Load and use a small HuggingFace model
- Generate simple text completions
- Apply LLMs to geoscience definition tasks

## 1. Understanding Tokens

**Tokens** are the basic units that language models work with. Text is broken down into tokens before being processed by the model. A token can be:
- A whole word (e.g., "seismic")
- Part of a word (e.g., "seis", "mic")
- Punctuation marks
- Special symbols

Let's see how tokenization works with a geoscience example:

In [ ]:
# TODO: Use utils library
def bert_tokenize_and_color(text, tokenizer ):
    colored_text = ""
    colors = ['#FF5733', '#33FF57', '#3357FF', '#FFD700', '#00CED1', '#FF00FF', '#FFFF00',
              '#FF0000', '#00FF00', '#0000FF', '#00FFFF', '#FF1493', '#8A2BE2',
              '#FF8C00', '#228B22', '#DC143C', '#32CD32', '#1E90FF', '#FFD700', '#FF69B4']

    tokens = tokenizer.tokenize(text)
    colored_html = ""
    
    for i, token in enumerate(tokens):
        color = colors[i % len(colors)]
        # Replace special characters for display
        display_token = token.replace('Ġ', '▁')  # GPT-2 uses Ġ for spaces
        colored_html += f'<span style="background-color:{color}; color: white; padding: 2px 4px; margin: 1px; border-radius: 3px;">{display_token}</span>'
    
    print(f"Original text: {text}")
    print(f"Number of tokens: {len(tokens)}")
    display(HTML(colored_html))
    print(f"Tokens: {tokens}")
    print("-" * 80)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Geoscience text examples
text = [
    "Seismic inversion is a geophysical technique.",
    "Hydrocarbon exploration uses seismic surveys.",
    "Reservoir characterization involves petrophysical analysis.",
    "What is the porosity and permeability of this formation?"
]

bert_tokenize_and_color(text, tokenizer)